<a href="https://colab.research.google.com/github/dstrick17/stock-market-and-job-postings/blob/main/data_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Engineering Project

In [ ]:
!pip install requests azure-storage-blob schedule
!pip install azure-storage-blob
!pip install azure-storage-blob



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.4/408.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 13.7 MB/s eta 0:00:00


In [ ]:
import requests
import json
from datetime import datetime
from azure.storage.blob import BlobServiceClient
import schedule
import time

# Adzuna API setup
APP_ID = "996295ea"  # Your Adzuna App ID
APP_KEY = "5c49257c76f351dee2315f0620b1ef17"  # Your Adzuna API Key
url = "http://api.adzuna.com/v1/api/jobs/us/search/1"

# Azure Storage Setup
account_url = "https://ds562storageacc.blob.core.windows.net"  # Your Azure account URL
container_name = "bronze"  # Store data in the 'bronze' container
blob_service_client = BlobServiceClient(account_url=account_url, credential="your_account_credential")  # Replace with your Azure account credentials

# Job Titles and Sectors (Adjust as needed)
job_titles = ["Data Scientist", "Software Engineer", "Database Administrator"]
sectors = ["Technology", "Healthcare", "Finance"]

# Function to fetch data from Adzuna API
def fetch_jobs(job_title):
    params = {
        "app_id": APP_ID,
        "app_key": APP_KEY,
        "results_per_page": 100,
        "content-type": "application/json",
        "what": job_title
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        jobs = response.json().get("results", [])
        print(f"Fetched {len(jobs)} jobs for {job_title}!\n")
        return jobs
    else:
        print(f"Error fetching jobs for {job_title}: {response.status_code}")
        return []

# Function to process job data
def process_job_data(jobs, sector):
    processed_data = []
    for job in jobs:
        job_info = {
            "Scrapped Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "Company": job.get("company", {}).get("display_name", "Unknown"),
            "Job Title": job.get("title", "Unknown"),
            "Salary": job.get("salary_max", "Not listed"),
            "Location": job.get("location", {}).get("display_name", "Unknown"),
            "Sector": sector
        }
        processed_data.append(job_info)
    return processed_data

# Function to upload data to Azure Blob Storage
def upload_to_azure(job_data):
    blob_name = f"job_data_{datetime.now().strftime('%Y%m%d%H%M%S')}.json"
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    json_data = json.dumps(job_data)
    blob_client.upload_blob(json_data, overwrite=True)
    print(f"Data uploaded to Azure Blob Storage with name {blob_name}")

# Function to fetch, process, and upload data every 2 hours
def fetch_and_upload():
    for job_title in job_titles:
        for sector in sectors:
            print(f"Fetching data for {job_title} in {sector}...")
            jobs = fetch_jobs(job_title)
            if jobs:
                processed_data = process_job_data(jobs, sector)
                upload_to_azure(processed_data)

# Schedule the task to run every 2 hours
schedule.every(2).hours.do(fetch_and_upload)

# Run the scheduled task
while True:
    schedule.run_pending()
    time.sleep(1)  # Wait for a second before checking the schedule again


Fetching data for Data Scientist in Technology...
Fetched 50 jobs for Data Scientist!



AzureSigningError: Invalid base64-encoded string: number of data characters (21) cannot be 1 more than a multiple of 4

## Adzuna Job API

APP_ID = "996295ea"
APP_KEY = "5c49257c76f351dee2315f0620b1ef17"

### Current Job Data

In [ ]:
import requests
from datetime import datetime

# Adzuna API setup
APP_ID = "996295ea"
APP_KEY = "5c49257c76f351dee2315f0620b1ef17"
url = "http://api.adzuna.com/v1/api/jobs/us/search/1"

params = {
    "app_id": APP_ID,
    "app_key": APP_KEY,
    "results_per_page": 100,
    "content-type": "application/json",
    "what": "software engineer",
}

# Fetch jobs
response = requests.get(url, params=params)

if response.status_code == 200:
    jobs = response.json().get("results", [])
    print(f"Fetched {len(jobs)} jobs successfully!\n")
else:
    print("Error:", response.status_code)
    exit()

# Process first 5 jobs
for i,job in enumerate(jobs[:100]):
    print(i)
    title = job.get("title", "N/A")
    company = job.get("company", {}).get("display_name", "N/A")

    location = job.get("location", {})
    area = location.get("area", [])

    # Extract city, state, and country properly
    city = area[-1] if len(area) >= 1 else "Unknown"
    state = area[1] if len(area) >= 2 else "Unknown"
    country = area[0] if len(area) >= 1 else "Unknown"

    salary = job.get("salary_max", "Not listed")

    # Convert job posting date to readable format
    posted_timestamp = job.get("created", None)
    if posted_timestamp:
        try:
            posted_date = datetime.strptime(posted_timestamp, "%Y-%m-%dT%H:%M:%SZ").strftime('%Y-%m-%d')
        except ValueError:
            posted_date = "Unknown"
    else:
        posted_date = "Unknown"

    print(f"Job Title: {title}")
    print(f"Company: {company}")
    print(f"Location: {city}, {state}, {country}")
    print(f"Salary: ${salary}")
    print(f"Posted On: {posted_date}")
    print("-" * 40)


Error: 503
0
Job Title: Manager, Software Engineering
Company: quantum-health
Location: Dublin, Ohio, US
Salary: $148536.86
Posted On: 2025-01-28
----------------------------------------
1
Job Title: Senior Embedded Software Engineer
Company: Johnson Controls
Location: New Freedom, Pennsylvania, US
Salary: $106522.63
Posted On: 2025-03-01
----------------------------------------
2
Job Title: Principal Software Engineer
Company: SNC
Location: Lonetree, Colorado, US
Salary: $155075.56
Posted On: 2025-02-19
----------------------------------------
3
Job Title: Software Engineer III
Company: SNC
Location: Lonetree, Colorado, US
Salary: $108648.05
Posted On: 2025-03-01
----------------------------------------
4
Job Title: Sr Software Engineer
Company: SNC
Location: Parker, Texas, US
Salary: $123560.88
Posted On: 2025-03-01
----------------------------------------
5
Job Title: Sr Software Engineer
Company: SNC
Location: Lonetree, Colorado, US
Salary: $129998.24
Posted On: 2025-03-01
--------

In [ ]:
import requests
from datetime import datetime

# Adzuna API setup
APP_ID = "996295ea"
APP_KEY = "5c49257c76f351dee2315f0620b1ef17"
url = "http://api.adzuna.com/v1/api/jobs/us/search/1"

params = {
    "app_id": APP_ID,
    "app_key": APP_KEY,
    "results_per_page": 100,
    "content-type": "application/json",
    "what": "data science",
}

# Fetch jobs
response = requests.get(url, params=params)

if response.status_code == 200:
    jobs = response.json().get("results", [])
    print(f"Fetched {len(jobs)} jobs successfully!\n")
else:
    print("Error:", response.status_code)
    exit()

# Process first 5 jobs
for i,job in enumerate(jobs[:100]):
    print(i)
    title = job.get("title", "N/A")
    company = job.get("company", {}).get("display_name", "N/A")

    location = job.get("location", {})
    area = location.get("area", [])

    # Extract city, state, and country properly
    city = area[-1] if len(area) >= 1 else "Unknown"
    state = area[1] if len(area) >= 2 else "Unknown"
    country = area[0] if len(area) >= 1 else "Unknown"

    salary = job.get("salary_max", "Not listed")

    # Convert job posting date to readable format
    posted_timestamp = job.get("created", None)
    if posted_timestamp:
        try:
            posted_date = datetime.strptime(posted_timestamp, "%Y-%m-%dT%H:%M:%SZ").strftime('%Y-%m-%d')
        except ValueError:
            posted_date = "Unknown"
    else:
        posted_date = "Unknown"

    print(f"Job Title: {title}")
    print(f"Company: {company}")
    print(f"Location: {city}, {state}, {country}")
    print(f"Salary: ${salary}")
    print(f"Posted On: {posted_date}")
    print("-" * 40)

Fetched 50 jobs successfully!

0
Job Title: Asset Servicing - Vice President, Data Science Project Lead
Company: BNY
Location: Bloomfield, Pennsylvania, US
Salary: $143097.38
Posted On: 2025-03-01
----------------------------------------
1
Job Title: Data Science Intern
Company: Maze Therapeutics
Location: South San Francisco, California, US
Salary: $86326.55
Posted On: 2025-02-22
----------------------------------------
2
Job Title: Senior Data Science Engineer
Company: SRI Tech Solutions
Location: Littleton, New Hampshire, US
Salary: $95838.15
Posted On: 2025-02-28
----------------------------------------
3
Job Title: Director of Finance and Administration, Biomedical Data Science
Company: Stanford University
Location: Stanford, California, US
Salary: $159399.05
Posted On: 2025-02-18
----------------------------------------
4
Job Title: Summer 2025 Intern - Data Science
Company: Stryker
Location: Portage, Michigan, US
Salary: $73863.23
Posted On: 2024-09-04
--------------------------

### Historical Job Data

In [ ]:
import requests
from datetime import datetime, timedelta

# Adzuna API setup
APP_ID = "996295ea"
APP_KEY = "5c49257c76f351dee2315f0620b1ef17"
url = "http://api.adzuna.com/v1/api/jobs/us/search/1"

# Get today's date and the date 30 days ago
end_date = datetime.utcnow()
start_date = end_date - timedelta(days=30)

# Format dates as strings in 'YYYY-MM-DD' format
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

params = {
    "app_id": APP_ID,
    "app_key": APP_KEY,
    "results_per_page": 100,
    "content-type": "application/json",
    "what": "data scientist",
}

# Fetch jobs
response = requests.get(url, params=params)

if response.status_code == 200:
    jobs = response.json().get("results", [])
    print(f"Fetched {len(jobs)} jobs successfully!\n")
else:
    print("Error:", response.status_code)
    exit()  # Exit the script if the API request failed

# Filter jobs for the last 30 days
job_postings_by_date = {}

for job in jobs:
    posted_timestamp = job.get("created", None)
    if posted_timestamp:
        try:
            posted_date = datetime.strptime(posted_timestamp, "%Y-%m-%dT%H:%M:%SZ").strftime('%Y-%m-%d')
        except ValueError:
            posted_date = "Unknown"
    else:
        posted_date = "Unknown"

    # Only include jobs posted in the last 30 days
    if posted_date >= start_date_str and posted_date <= end_date_str:
        if posted_date not in job_postings_by_date:
            job_postings_by_date[posted_date] = []

        # Limit to 1 job per day
        if len(job_postings_by_date[posted_date]) < 1:
            job_postings_by_date[posted_date].append(job)

# Print one job posting per day for the last 30 days
for date in sorted(job_postings_by_date.keys(), reverse=True):  # Sort by most recent date
    job = job_postings_by_date[date][0]  # Get the first job for that date
    title = job.get("title", "N/A")
    company = job.get("company", {}).get("display_name", "N/A")
    location = job.get("location", {})
    area = location.get("area", [])

    # Extract city, state, and country properly
    city = area[-1] if len(area) >= 1 else "Unknown"
    state = area[1] if len(area) >= 2 else "Unknown"
    country = area[0] if len(area) >= 1 else "Unknown"

    salary = job.get("salary_max", "Not listed")

    # Print job details
    print(f"\nJob Postings on {date}:")
    print(f"Job Title: {title}")
    print(f"Company: {company}")
    print(f"Location: {city}, {state}, {country}")
    print(f"Salary: ${salary}")
    print(f"Posted On: {posted_date}")
    print("-" * 40)

Fetched 50 jobs successfully!


Job Postings on 2025-03-01:
Job Title: Data Scientist
Company: BNY
Location: Bloomfield, Pennsylvania, US
Salary: $106251.63
Posted On: 2025-03-01
----------------------------------------

Job Postings on 2025-02-27:
Job Title: Associate Data Scientist
Company: NextEra
Location: North Palm Beach, Florida, US
Salary: $99214.85
Posted On: 2025-03-01
----------------------------------------

Job Postings on 2025-02-26:
Job Title: Data Scientist/Data Architect
Company: Breakforth Solutions Inc
Location: Mc Lean, Virginia, US
Salary: $113617.8
Posted On: 2025-03-01
----------------------------------------

Job Postings on 2025-02-25:
Job Title: Data Scientist
Company: Verizon
Location: Ashburn, Virginia, US
Salary: $175098.33
Posted On: 2025-03-01
----------------------------------------

Job Postings on 2025-02-22:
Job Title: Sr. Data Scientist
Company: Solstice Consulting LLC
Location: Schiller Park, Illinois, US
Salary: $114368.23
Posted On: 2025-03-01
---

## Alpha Vantage (Stock MarkeT) API Key

APIKEy: QUGD4YMIKXMCP391

In [ ]:
import requests

# Dictionary of tech stock symbols and corresponding company names
tech_stocks = {
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corporation",
    "GOOGL": "Alphabet Inc. (Google)",
    "AMZN": "Amazon.com Inc.",
    "TSLA": "Tesla Inc.",
    "NVDA": "NVIDIA Corporation",
    "FB": "Meta Platforms, Inc. (formerly Facebook)",
    "INTC": "Intel Corporation",
    "CSCO": "Cisco Systems, Inc.",
    "AMD": "Advanced Micro Devices, Inc.",
}

# Alpha Vantage API setup
api_key = "QUGD4YMIKXMCP391"

# Loop through each tech stock symbol
for symbol, company_name in tech_stocks.items():
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=5min&apikey={api_key}"

    # Fetch data for each stock
    r = requests.get(url)
    data = r.json()

    # Extract the latest time series data
    time_series_key = "Time Series (5min)"
    if time_series_key in data:
        latest_timestamp = max(data[time_series_key].keys())  # Get the latest timestamp
        latest_data = data[time_series_key][latest_timestamp]

        # Extract stock values
        open_price = latest_data["1. open"]
        high_price = latest_data["2. high"]
        low_price = latest_data["3. low"]
        close_price = latest_data["4. close"]
        volume = latest_data["5. volume"]

        # Print results for each stock with company name
        print(f"Company: {company_name} ({symbol})")
        print(f"Timestamp: {latest_timestamp}")
        print(f"Open: {open_price}")
        print(f"High: {high_price}")
        print(f"Low: {low_price}")
        print(f"Close: {close_price}")
        print(f"Volume: {volume}")
        print("-" * 40)
    else:
        print(f"Error: Unable to fetch data for {symbol} ({company_name}).")




Company: Apple Inc. (AAPL)
Timestamp: 2025-02-28 19:55:00
Open: 241.9500
High: 241.9600
Low: 241.6100
Close: 241.6100
Volume: 4364
----------------------------------------
Company: Microsoft Corporation (MSFT)
Timestamp: 2025-02-28 19:55:00
Open: 396.6450
High: 396.7900
Low: 396.3000
Close: 396.3000
Volume: 2394
----------------------------------------
Company: Alphabet Inc. (Google) (GOOGL)
Timestamp: 2025-02-28 19:55:00
Open: 170.6900
High: 170.7100
Low: 170.5000
Close: 170.6100
Volume: 3737
----------------------------------------
Company: Amazon.com Inc. (AMZN)
Timestamp: 2025-02-28 19:55:00
Open: 212.6750
High: 212.7000
Low: 212.5500
Close: 212.6250
Volume: 3195
----------------------------------------
Company: Tesla Inc. (TSLA)
Timestamp: 2025-02-28 19:55:00
Open: 293.6800
High: 294.4500
Low: 293.5000
Close: 294.4500
Volume: 43158
----------------------------------------
Company: NVIDIA Corporation (NVDA)
Timestamp: 2025-02-28 19:55:00
Open: 124.8900
High: 125.0000
Low: 124.8600


###  Historical Stock Data


In [ ]:
import requests

# Dictionary of tech stock symbols and corresponding company names
tech_stocks = {
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corporation",
    "GOOGL": "Alphabet Inc. (Google)",
    "AMZN": "Amazon.com Inc.",
    "TSLA": "Tesla Inc.",
    "NVDA": "NVIDIA Corporation",
    "FB": "Meta Platforms, Inc. (formerly Facebook)",
    "INTC": "Intel Corporation",
    "CSCO": "Cisco Systems, Inc.",
    "AMD": "Advanced Micro Devices, Inc.",
}

# Alpha Vantage API setup
api_key = "QUGD4YMIKXMCP391"

# Loop through each tech stock symbol and retrieve historical data
for symbol, company_name in tech_stocks.items():
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}"

    # Fetch historical data for each stock
    r = requests.get(url)
    data = r.json()

    # Extract the daily time series data
    time_series_key = "Time Series (Daily)"
    if time_series_key in data:
        print(f"\nCompany: {company_name} ({symbol})")
        # Loop through the dates in the historical data
        for date, stock_data in data[time_series_key].items():
            open_price = stock_data["1. open"]
            high_price = stock_data["2. high"]
            low_price = stock_data["3. low"]
            close_price = stock_data["4. close"]
            volume = stock_data["5. volume"]

            # Display the historical stock data for each date
            print(f"Date: {date}")
            print(f"Open: {open_price}")
            print(f"High: {high_price}")
            print(f"Low: {low_price}")
            print(f"Close: {close_price}")
            print(f"Volume: {volume}")
            print("-" * 40)
    else:
        print(f"Error: Unable to fetch historical data for {symbol} ({company_name}).")



Company: Apple Inc. (AAPL)
Date: 2025-02-28
Open: 236.9500
High: 242.0900
Low: 230.2000
Close: 241.8400
Volume: 56833360
----------------------------------------
Date: 2025-02-27
Open: 239.4100
High: 242.4600
Low: 237.0600
Close: 237.3000
Volume: 41153639
----------------------------------------
Date: 2025-02-26
Open: 244.3300
High: 244.9800
Low: 239.1300
Close: 240.3600
Volume: 44433564
----------------------------------------
Date: 2025-02-25
Open: 248.0000
High: 250.0000
Low: 244.9100
Close: 247.0400
Volume: 48013272
----------------------------------------
Date: 2025-02-24
Open: 244.9250
High: 248.8600
Low: 244.4200
Close: 247.1000
Volume: 51326396
----------------------------------------
Date: 2025-02-21
Open: 245.9500
High: 248.6900
Low: 245.2200
Close: 245.5500
Volume: 53197431
----------------------------------------
Date: 2025-02-20
Open: 244.9400
High: 246.7800
Low: 244.2900
Close: 245.8300
Volume: 32316907
----------------------------------------
Date: 2025-02-19
Open: 244.

In [ ]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=9BMYB8D9X97EPZVI&month=2009-01'
r = requests.get(url)
data = r.json()

print(data)

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'IBM', '3. Last Refreshed': '2025-02-28', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-02-28': {'1. open': '250.8550', '2. high': '252.8099', '3. low': '246.5400', '4. close': '252.4400', '5. volume': '7988809'}, '2025-02-27': {'1. open': '255.2200', '2. high': '257.6300', '3. low': '253.0500', '4. close': '253.2300', '5. volume': '3402234'}, '2025-02-26': {'1. open': '258.1000', '2. high': '258.3250', '3. low': '254.4104', '4. close': '255.8400', '5. volume': '3460124'}, '2025-02-25': {'1. open': '261.0800', '2. high': '263.4800', '3. low': '256.7700', '4. close': '257.7500', '5. volume': '6292487'}, '2025-02-24': {'1. open': '261.5000', '2. high': '263.8450', '3. low': '259.5800', '4. close': '261.8700', '5. volume': '4398107'}, '2025-02-21': {'1. open': '263.8450', '2. high': '264.8300', '3. low': '261.1000', '4. close': '261.4800', '5. volu

In [ ]:
import json
print(json.dumps(data, indent=4))

{
    "Meta Data": {
        "1. Information": "Daily Prices (open, high, low, close) and Volumes",
        "2. Symbol": "IBM",
        "3. Last Refreshed": "2025-02-28",
        "4. Output Size": "Compact",
        "5. Time Zone": "US/Eastern"
    },
    "Time Series (Daily)": {
        "2025-02-28": {
            "1. open": "250.8550",
            "2. high": "252.8099",
            "3. low": "246.5400",
            "4. close": "252.4400",
            "5. volume": "7988809"
        },
        "2025-02-27": {
            "1. open": "255.2200",
            "2. high": "257.6300",
            "3. low": "253.0500",
            "4. close": "253.2300",
            "5. volume": "3402234"
        },
        "2025-02-26": {
            "1. open": "258.1000",
            "2. high": "258.3250",
            "3. low": "254.4104",
            "4. close": "255.8400",
            "5. volume": "3460124"
        },
        "2025-02-25": {
            "1. open": "261.0800",
            "2. high": "263.48